In [55]:
import os
import sys
import duckdb
import re
# sys.path.append('../../')
sys.path.append(rf"C:\Git\BI0730")
from libs.geral.utils import * 
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
user = os.getlogin()

In [56]:

con = duckdb.connect()

path = rf"C:\Users\{user}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\cia_pcp_indicador_principalidade_historico\\2025*.parquet"
query = rf'''
    CREATE TABLE dados AS
    SELECT * FROM read_parquet('{path}')
    
   
'''

con.execute(query)

In [57]:

query = f''' 
      WITH tratados AS 
        (
            SELECT *,
            ROW_NUMBER() OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes DESC) AS rn,
            pontos_principalidade - LAG(pontos_principalidade)  OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC) AS var_pontos

            FROM dados
            QUALIFY rn <= 4
            ORDER BY ano_Mes
        ),
        
        marcacoes AS (
            SELECT  
            *,
            CASE WHEN var_pontos < 0 THEN 1 ELSE 0 END AS queda_flag
            FROM tratados 
            WHERE rn <= 3
            
        ),
        checagem AS (
            SELECT *,
            SUM(queda_flag) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes) AS soma_quedas
            FROM marcacoes
        )
        
        SELECT  * FROM checagem WHERE soma_quedas >= 3 and ano_mes = (SELECT MAX(ano_mes) FROM checagem)
        --SELECT  * FROM checagem WHERE cpf_cnpj = '00362741948'
        

        '''
con.sql(query).to_df()
       

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,ano_mes,cpf_cnpj,segmento,cod_cooperativa,cod_central,cod_agencia,cod_carteira,status_associado,porte_padrao,meses_desde_associacao,assoc_desde,nivel_risco,des_origem,vlr_capital_social,idade,pix_trans_30d,cad_pix,sum_titulo_investimento,mobi_transacionou_30d,sum_previdencia,sum_investimentos,total_valor_cartao_mercado_scr,total_valor_cartao_sicredi,total_valor_agro_sicredi,total_valor_agro_mercado_scr,total_valor_outros_sicredi,total_valor_agro_mercado_cpr,sow_agro,total_valor_outros_mercado_scr,sow_outros,sow_cartao,possui_cartao_credito,debito_conta_ativo,possui_domicilio,possui_cartao_debito,possui_folha_pagamento,possui_cobranca,possui_consorcio_motos,possui_adquirencia,possui_consorcio_pesados,possui_consorcio_servicos,possui_consorcio_automoveis,possui_consorcio_imoveis,possui_seguro_rural,possui_previdencia,possui_seguro_residencial,possui_seguro_agricola,possui_seguro_patrimonial,possui_seguro_vida,possui_seguro_automovel,possui_conta_salario,transacao_app,flg_ativo,flg_novo_assoc,flg_idade_maior_18,cad_pix_ativo,ativou_open_finance,cash_in,cash_out,cash_total_fator,cash_total,pontos_produtos_basicos,pontos_rural,pontos_planejamento,pontos_protecao,pontos_previdencia,pontos_movimentacao,pontos_cred_cartao,pontos_cred_outros,pontos_investimento,pontos_cred_rural,flag_principalidade,pontos_principalidade,faixa_atingimento_meta,percentual_atingimento,dat_carga_incremental,faixa_categoria,etl_created_on,etl_updated_on,data_insercao,data_atualizacao,filtro_ano_mes_inicial,filtro_ano_mes_final,rn,var_pontos,queda_flag,soma_quedas
0,202506,00546375979,PF,0730,7000,18,141,ATIVO,PF IV,37,202205,BAIXO 1,LEGADO,"1,430.06",45,S,S,0.00,N,0.00,"1,430.06","295,641.09","23,918.91",0.00,0.00,0.01,0.00,0.00,"514,957.56",0.00,0.08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0.00,"5,496.87",0.00,"5,496.87",8,0,0,0,0,18,4,0,0,0,0,30,20-80%,23.076923076923077,2025-07-17,Emergente,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-4,1,3.00
1,202506,00608787973,PF,0730,7000,04,111,ATIVO,PF II,59,202007,MÉDIO 1,LEGADO,"1,251.63",51,S,S,"2,602.44",S,0.00,"3,854.07","4,108.59","4,108.59",0.00,0.00,0.00,0.00,0.00,"8,153.83",0.00,1.00,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,"3,462.91","3,426.25",0.00,"6,889.16",20,0,0,0,0,34,30,0,19,0,1,103,120+,128.75,2025-07-17,Maduro,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-14,1,3.00
2,202506,00672536951,PF,0730,7000,11,010101,ATIVO,PF II,18,202312,ALTÍSSIMO,DIGITAL,0.00,45,S,S,0.00,S,0.00,0.00,0.00,0.00,0.00,0.00,"3,843.71",0.00,0.00,"50,394.99",0.08,0.00,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,"5,540.00","5,252.21",0.00,"10,792.21",20,0,0,0,0,53,0,2,0,0,0,75,80-100%,93.75,2025-07-17,Promissor,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-2,1,3.00
3,202506,00761059709,PF,0730,7000,04,111,ATIVO,PF I,60,202006,ALTO 1,LEGADO,"1,557.56",55,S,S,0.73,S,0.00,"1,558.29","5,853.53","5,853.53",0.00,0.00,"1,333.95",0.00,0.00,"2,092.72",0.64,1.00,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,275.00,"2,888.99",0.00,"3,163.99",20,0,0,0,0,21,20,13,10,0,1,84,120+,168.0,2025-07-17,Maduro,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-4,1,3.00
4,202506,00779699955,PF,0730,7000,31,131,ATIVO,PF III,34,202208,BAIXÍSSIMO,LEGADO,998.38,41,S,S,"45,325.56",S,0.00,"46,323.94",933.67,933.67,0.00,0.00,0.00,0.00,0.00,37.06,0.00,1.00,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,"4,055.00","5,951.01",0.00,"10,006.01",16,0,10,0,0,40,40,0,61,0,1,167,120+,167.0,2025-07-17,Maduro,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-36,1,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [58]:
saida = rf"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\associados_queda_principalidade.parquet"
tmp = saida + ".tmp"

# Executar a CTE (usa a variável 'query' já definida acima) e materializar
df_quedas = con.sql(query).fetchdf()

# Validar presença das colunas calculadas
esperadas_calc = ['var_pontos','queda_flag','soma_quedas']
faltan = [c for c in esperadas_calc if c not in df_quedas.columns]
if faltan:
    raise RuntimeError(f"Faltan columnas calculadas: {faltan}")

# Guardar atomicamente
df_quedas.to_parquet(tmp, index=False)
os.replace(tmp, saida)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [59]:
df = pd.read_parquet(rf"C:\\Users\\{USER}\\Sicredi\\TimeBI_0730 - Documentos\\01_Rotineiros\\33_GiroCarteira\\giro_de_carteira\\associados_queda_principalidade.parquet")
df.head()

,ano_mes,cpf_cnpj,segmento,cod_cooperativa,cod_central,cod_agencia,cod_carteira,status_associado,porte_padrao,meses_desde_associacao,assoc_desde,nivel_risco,des_origem,vlr_capital_social,idade,pix_trans_30d,cad_pix,sum_titulo_investimento,mobi_transacionou_30d,sum_previdencia,sum_investimentos,total_valor_cartao_mercado_scr,total_valor_cartao_sicredi,total_valor_agro_sicredi,total_valor_agro_mercado_scr,total_valor_outros_sicredi,total_valor_agro_mercado_cpr,sow_agro,total_valor_outros_mercado_scr,sow_outros,sow_cartao,possui_cartao_credito,debito_conta_ativo,possui_domicilio,possui_cartao_debito,possui_folha_pagamento,possui_cobranca,possui_consorcio_motos,possui_adquirencia,possui_consorcio_pesados,possui_consorcio_servicos,possui_consorcio_automoveis,possui_consorcio_imoveis,possui_seguro_rural,possui_previdencia,possui_seguro_residencial,possui_seguro_agricola,possui_seguro_patrimonial,possui_seguro_vida,possui_seguro_automovel,possui_conta_salario,transacao_app,flg_ativo,flg_novo_assoc,flg_idade_maior_18,cad_pix_ativo,ativou_open_finance,cash_in,cash_out,cash_total_fator,cash_total,pontos_produtos_basicos,pontos_rural,pontos_planejamento,pontos_protecao,pontos_previdencia,pontos_movimentacao,pontos_cred_cartao,pontos_cred_outros,pontos_investimento,pontos_cred_rural,flag_principalidade,pontos_principalidade,faixa_atingimento_meta,percentual_atingimento,dat_carga_incremental,faixa_categoria,etl_created_on,etl_updated_on,data_insercao,data_atualizacao,filtro_ano_mes_inicial,filtro_ano_mes_final,rn,var_pontos,queda_flag,soma_quedas
0,202506,00355679965,PF,0730,7000,33,141,ATIVO,PF IV,32,202210,BAIXÍSSIMO,LEGADO,863.15,45,S,S,0.00,N,0.00,863.15,"74,831.14","4,876.56",0.00,0.00,"14,168.99",0.00,0.00,"769,383.00",0.02,0.07,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0.00,"3,367.35",0.00,"3,367.35",16,0,0,0,0,11,3,0,0,0,0,30,20-80%,23.076923076923077,2025-07-17,Emergente,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-2,1,3.00
1,202506,00403761921,PF,0730,7000,32,131,ATIVO,PF III,12,202406,BAIXÍSSIMO,LEGADO,277.81,65,N,S,"42,614.59",S,0.00,"42,892.40","12,633.76","4,249.86",0.00,0.00,"1,726.91",0.00,0.00,"106,500.17",0.02,0.34,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0.00,"1,863.93",0.00,"1,863.93",12,0,0,0,0,7,13,0,57,0,0,89,80-100%,89.0,2025-07-17,Promissor,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-3,1,3.00
2,202506,00427605938,PF,0730,7000,21,121,ATIVO,PF II,54,202012,BAIXO 2,LEGADO,"1,174.00",44,S,S,199.39,S,0.00,"1,373.39","3,294.48",0.00,0.00,0.00,"3,453.51",0.00,0.00,"16,729.60",0.21,0.00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0.00,528.43,0.00,528.43,15,0,0,0,0,2,0,6,6,0,0,29,20-80%,36.25,2025-07-17,Emergente,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-6,1,3.00
3,202506,00732074975,PF,0730,7000,27,141,ATIVO,PF IV,74,201904,ALTO 2,LEGADO,21.00,44,N,S,0.16,N,0.00,21.16,"13,832.09",0.00,0.00,0.00,"173,530.56",0.00,0.00,"173,530.56",1.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,50,0,0,0,50,20-80%,38.46153846153847,2025-07-17,Emergente,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-1,1,3.00
4,202506,00772480907,PF,0730,7000,34,141,ATIVO,PF IV,51,202103,BAIXO 2,LEGADO,"1,352.03",45,N,N,0.00,N,0.00,"1,352.03","666,463.00","9,093.47",0.00,0.00,0.00,0.00,0.00,"3,583,053.25",0.00,0.01,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,600.00,"2,097.66",0.00,"2,697.66",4,0,0,0,0,8,0,0,0,0,0,12,0-20%,9.230769230769232,2025-07-17,Aspirante,2025-07-17 19:21:28.324,2025-07-17 19:21:28.324,2025-08-07,2025-08-07,202506,202506,1,-13,1,3.00


In [60]:
assodia =  carregar_associados_completo()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [61]:
assodia

,cod_agencia,cod_carteira,cpf_cnpj,num_conta,nom_associado,vlr_renda_mensal,isa,mc_total,cbo_ou_cnae,tipo_pessoa,flg_correntista,flg_ativo,dt_ult_movimento,dt_ini_asso,des_segmento,des_subsegmento,des_publico_estrategico,des_faixa_risco,des_marca,data_competencia,dat_criacao_registro,flg_carteira_ativa,cod_conglomerado_economico,des_conglomerado_economico,comercial,celular,telefone,dt_renovacao_cadastral,agcart,gestor,gestor_ldap,'GA?',cad_vencido
0,05,111,04232508929,017566,JUSSARA BARBOSA,"2,200.00",5.00,112.62,Conselheiro tutelar,PF,S,S,2025-08-13,2001-10-04,PF,PF II,DEMAIS,MÉDIO 2,SICREDI,2025-08-18,2025-08-18,S,16100,GRUPO JUSSARA BARBOSA TRANSPORTES,None,42999231418,(42) 99923.1418,2024-12-23 00:00:00,05111,Mislaine Martins Carneiro,mislaine_carneiro,GA?,0
1,24,106,11153244977,089091,JOCINEIA MORAES TOGESKI,"2,682.69",1.00,0.07,Assistente administrativo,PF,S,N,2023-10-09,2023-10-09,PF,PF II,DEMAIS,BAIXO 1,SICREDI,2025-08-18,2025-08-18,S,None,None,None,42998665177,(42) 99866.5177,2025-06-25 05:21:46,None,None,None,GA?,0
2,50,010101,26278555816,100144,ADJAILSON DJAIN PEREIRA DA SILVA,"14,838.00",10.00,-40.30,ADMINISTRADOR,PF,S,S,2025-08-19,2024-11-22,PF,PF I,DEMAIS,MÉDIO 2,FISITAL,2025-08-18,2025-08-18,S,None,None,None,13981122022,(13) 98112.2022,2025-07-02 00:00:00,5010101,Samuel De Oliveira,samueloliveira,GA?,0
3,39,010301,30593017862,100273,RAPHAEL TORQUATO SOBREIRA,"7,975.61",6.00,152.02,ADMINISTRADOR,PF,S,S,2025-08-18,2024-11-22,PF,PF III,DEMAIS,BAIXO 2,FISITAL,2025-08-18,2025-08-18,S,None,None,None,11947983328,(11) 94798.3328,2025-06-28 00:00:00,3910301,Mariana Bevilacqua Teixeira Covalli,mariana_covalli,GA?,0
4,35,010401,11058055909,100485,HELENA HECK,0.01,3.00,1.74,ESTUDANTE,PF,S,S,2025-08-11,2024-11-22,PF,PF I,MENOR,ALTO 1,FISITAL,2025-08-18,2025-08-18,S,None,None,None,42999291980,(42) 99929.1980,2024-12-23 00:00:00,3510401,Luiz Gustavo Correia,luiz_gcorreia,GA?,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197714,32,010101,11521190950,374805,BRENNO KRISLEY MARQUES DE JESUS,"7,800.00",7.00,61.01,ASSISTENTE DE VENDAS,PF,S,S,2025-08-18,2022-08-26,PF,PF III,JOVEM,MÉDIO 2,FISITAL,2025-08-18,2025-08-18,S,None,None,None,41992093677,(41) 99209.3677,2025-05-17 00:00:00,3210101,Jordana Sampaio Da Silva,silva_jordana,GA?,0
197715,03,010101,01234500981,571631,FABIANO RIBEIRO LEMES PEREIRA,"1,370.28",0.00,2.04,Classificador de madeira,PF,S,N,2023-01-11,2021-12-21,PF,PF I,DEMAIS,ALTO 1,FISITAL,2025-08-18,2025-08-18,S,,,None,21971552947,(21) 97155.2947,2021-12-21 00:00:00,0310101,Daiane Scheiffer Oliveira,oliveira_daiane,GA?,1
197716,04,010202,06845233969,636504,FELIPE JOSE DO NASCIMENTO,"4,000.00",6.00,25.21,CAMINHONEIRO AUTÔNOMO (ROTAS REGIONAIS E INTERNACIONAIS),PF,S,S,2025-08-19,2023-07-11,PF,PF II,DEMAIS,ALTO 2,FISITAL,2025-08-18,2025-08-18,S,,N,None,42998736773,(42) 99873.6773,2025-06-12 00:00:00,0410202,Ariane Cristina Monteiro Bonnet,ariane_bonnet,GA?,0
197717,15,010102,46769079828,041893,RAFAELA DE OLIVEIRA ESTANISLAU,"1,600.00",4.00,0.98,Atendente de lojas e mercados,PF,S,S,2025-08-12,2024-06-06,PF,PF I,JOVEM,MÉDIO 2,FISITAL,2025-08-18,2025-08-18,S,,,None,13996888241,(13) 99688.8241,2024-05-29 09:40:45,1510102,Matheus Fernando Da Costa,matheus_fernando,GA?,1


In [62]:
# Ajuste prévio de ano_atual para evitar NameError e definir marca de data (AAAAMM)
from datetime import date

# Garante que ANO exista
try:
    ANO
except NameError:
    ANO = date.today().year

ano_atual = ANO

# Marca de data para uso em nomes de arquivos (AAAAMM)
data = date.today().strftime('%Y%m')

In [63]:
ANO = 2025
USER = os.getlogin()

def localizar_arquivo_principalidade(ano:int, base_parquet_dir:str,
                                     nome_base:str='cia_pcp_indicador_principalidade_historico'):
    """Retorna (caminho_escolhido, lista_arquivos_avaliados).
    Regras:
      1. Se existir diretório <base>/<nome_base>/ com vários parquets:
         - Considera arquivos *.parquet que contenham nome_base.
         - Extrai prefixo inicial AAAAMM antes do primeiro '_'. 
         - Primeiro tenta só os do ano solicitado (AAAAMM começa com str(ano)).
         - Se não houver para o ano, considera todos e pega o maior AAAAMM.
      2. Se não houver diretório, tenta arquivo único <base>/<nome_base>.parquet
    """
    dir_candidatos = os.path.join(base_parquet_dir, nome_base)
    avaliados = []
    selecionaveis = []
    if os.path.isdir(dir_candidatos):
        for f in os.listdir(dir_candidatos):
            if f.lower().endswith('.parquet') and nome_base in f:
                avaliados.append(f)
                m = re.match(r'^(\d{6})_', f)
                if m:
                    aaaamm = m.group(1)
                    try:
                        selecionaveis.append((int(aaaamm), f))
                    except ValueError:
                        pass
        # Filtrar por ano solicitado
        ano_prefix = str(ano)
        por_ano = [t for t in selecionaveis if str(t[0]).startswith(ano_prefix)]
        if por_ano:
            por_ano.sort()
            escolhido = por_ano[-1][1]
        elif selecionaveis:
            selecionaveis.sort()
            escolhido = selecionaveis[-1][1]
        else:
            # Nenhum com padrão AAAAMM_ => fallback arquivo único
            escolhido = None
        if escolhido:
            return os.path.join(dir_candidatos, escolhido), avaliados
        else:
            # fallback arquivo único na raiz
            return os.path.join(base_parquet_dir, f'{nome_base}.parquet'), avaliados
    else:
        # Diretório não existe => assumir arquivo único
        return os.path.join(base_parquet_dir, f'{nome_base}.parquet'), avaliados

principalidade_path, lista_avaliados = localizar_arquivo_principalidade(ANO, PATH_BASES_PARQUET)
print('[principalidade] candidatos avaliados:', lista_avaliados[:8], '...' if len(lista_avaliados) > 8 else '')
print('[principalidade] selecionado:', principalidade_path)

RUTAS = {
    'inadimplentes': os.path.join(PATH_BASES_PARQUET, 'base_inadimplentes.parquet'),
    'associados_dir': os.path.join(PATH_BASES_PARQUET, 'associados_total_diario'),
    'saida_dir': fr'C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira',
    'saida_dir_giro': fr'C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira',
    'principalidade': principalidade_path
}

os.makedirs(RUTAS['saida_dir_giro'], exist_ok=True)

config = {
    'ano': ano_atual,
    'pasta_saida_duckdb': rf"C:\\Users\\{USER}\\Sicredi\\TimeBI_0730 - Documentos\\01_Rotineiros\\33_GiroCarteira\\giro_de_carteira\\associados_queda_principalidade.parquet",
    'pasta_isa_historico': rf"C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira",
    'arquivo_saida': rf"C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira_2025.parquet",
    'filtros_risco_bbm': ["BAIXÍSSIMO", "BAIXO 1", "BAIXO 2", "MÉDIO 1", "MÉDIO 2"],
    'dias_sem_movimentacao': (20, 45)
}
PARAMS = {
    'filtros_risco_bbm': ["BAIXÍSSIMO", "BAIXO 1", "BAIXO 2", "MÉDIO 1", "MÉDIO 2"],
    'dias_sem_movimentacao': (20, 45),
    'principalidade': ['sow']
}

for k, v in RUTAS.items():
    print(f'{k}: {v}')

[principalidade] candidatos avaliados: ['202301_cia_pcp_indicador_principalidade_historico.parquet', '202302_cia_pcp_indicador_principalidade_historico.parquet', '202303_cia_pcp_indicador_principalidade_historico.parquet', '202304_cia_pcp_indicador_principalidade_historico.parquet', '202305_cia_pcp_indicador_principalidade_historico.parquet', '202306_cia_pcp_indicador_principalidade_historico.parquet', '202307_cia_pcp_indicador_principalidade_historico.parquet', '202308_cia_pcp_indicador_principalidade_historico.parquet'] ...
[principalidade] selecionado: C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\cia_pcp_indicador_principalidade_historico\202506_cia_pcp_indicador_principalidade_historico.parquet
inadimplentes: C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\base_inadimplentes.parquet
associados_dir: C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\associados_total_diario
saida_dir: C:\Users\c

el archivo que sustenta es el archivo de principalidade, los merge serán realizados a este archivo para poder filtrar por associados sin inadimplencia y que no tengan uso en el campo 'mobi_transacionou_30d' ó ultimo movimiento entre 20 - 45 días

serán generados 2 parquet uno con toda la información y un dataframe construido con la siguientes campos e informaciones

{  
  "titulo": "string",    
  "usuarioResponsavel": "string", ----> GESTOR
  "usuarioSolicitante": "string", -----> 
  "numeroAgencia": 0, -----> cod_agencia
  "descricao": "string",  --------> descricao texto com detalhes e indices do associado + o texto da oferta de produtos a entregar
  "statusChamado": "RASCUNHO", 
  "departamentoId": 0,
  "nomeDepartamento": "string", --------->  _nom_agencia
  "dataNecessidade": "2025-08-14T11:45:45.991Z",
  "categoriaChamado": {
    "categoriaChamadoId": 0,  ---- > 1 ou 2
    "nomeCategoria": "string", ------> 
    "descricaoCategoria": "string", 
    "dataCriacao": "2025-08-14",
    "departamentoId": 0}
}

Titulo () + nome associado + cpf 

Descrição:
PF OU PJ
ISA atual 
ISA 3M
MC TOTAL ATUAL
MC 6M
INDICES PRINCIPALIDADE

OFERTA : TEXTO conformado por  categoria + descriçao categoria + produto  1 texto : 

  Oferta Acesso  ----->  Categoria 1 
(descriçao categoria) Ofertar Produtos/Serviços Básicos : {} ou [] {add. Produto}-------> Titulo + nome associado + cpf  
. Chave PIX ativa 
· Débito automático
· Cartão de débito
· Cartão de crédito
. Canais digitais
. Credenciamento ativo
· Cobrança
. Folha de Pagamento
. Domicílio
. Open Finance (receptor)

 Oferta Fluxo de Caixa   ----->  Categoria 2 
if % produto in range 
(descriçao categoria) Ofertar Movimentações : [] ou {} {add. Crédito &| cartão}-------> Titulo + nome associado + cpf
. Somatório do total de movimentações:
- Receber (Cash-in)
- Pagar (Cash-out)

Crédito - cartão
. SOW (Share of Wallet)
referente à cartão

Credito - outros
SOW (Share of Wallet)
referente à créditos



df

el campo categoria puede tener dos contenidos acesso y fluxo de caixa será utilizado como una flag de la condicion de los productos que no tiene 

el campo producto deberá conter todos los productos que no tiene haciendo un array para luego ser utilizado en la construccion del texto del campo descripcion tendremos 2 campos productos categoria I y productos categoria II


la categoria fluxo de caixa tiene percentuais que deberán ser utilizados con rangos para saber si existe posibilidad de un upgrade, se mantiene o que producto ofertar, esto deberá ser gravado en el campo produto ahora colocaremos solo los porcentages que existen en el sow, indicando a que corresponden, para esso utilizaremos el titulo del campo y e la cifra contenido en el valor del campo

al final de todo concatenaremos todos los textos de los campos 


Titulo () + nome associado + cpf 

Descrição:
PF OU PJ
ISA atual 
ISA 3M
MC TOTAL ATUAL
MC 6M
INDICES PRINCIPALIDADE

OFERTA : TEXTO conformado por  categoria + descriçao categoria + produto  1 texto 


y crearemos el campo Descripcion 


deberá ser generado un codigo que implemente filtros de indimplencia, risco bbm y que la ultima movimentacion dejó de entre 20 y 45
será construida el campo descripcion con las caracteristia de los datos existentes en los parquets utilizados según el modelo, si el associado NO tiene el producto será addicionado como producto en la descripción con el titulo del campo como catgroria 



las columnas para este último parquet o dataframe serán 
{  
  "titulo": "string",    
  "usuarioResponsavel": "string", ----> GESTOR
  "usuarioSolicitante": "string", -----> 
  "numeroAgencia": 0, -----> cod_agencia
  "descricao": "string",  --------> descricao texto com detalhes e indices do associado + o texto da oferta de produtos a entregar
  "statusChamado": "RASCUNHO", 
  "departamentoId": 0,
  "nomeDepartamento": "string", --------->  _nom_agencia
  "dataNecessidade": "2025-08-14T11:45:45.991Z", ----> exemplo
  "categoriaChamado": {
    "categoriaChamadoId": 0,  ---- > 1 ou 2
    "nomeCategoria": "string", ------> 
    "descricaoCategoria": "string", 
    "dataCriacao": "2025-08-14",
    "departamentoId": 0}
}


las bases son conformadas mensualmente por lo que utilizaremos una data base de mes pero dividremos el total en 4 partes, para que sea usado por semana, colocando los mas antiguos para ser contactados la primera semana y así sucesivamente hasta nuevamente ser generado un nuevo archivo para eso generaremos la informacion en el campo dataNecessidade



del dashboard associados total dashboard
utilizaremos cpf_cnpj e nome_agencia,mc_total_1,gestor
C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\associados_totais_tratados_dashboard.parquet

de isa historico

C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\isa_historico_analise_20250814.parquet utilizaremos 
gestor,
isa_status
isa_media
isa_202503
isa_202504
isa_202505
isa_202506  siempre utilizando los ultimos 3 

de acuerdo a todo lo dicho necesito que generes un ipynb de debug con las rutas y las condiciones generadas,  utilizaremos pandas numpy re os sys 

In [64]:
# Colunas arquivo principalidade
colunas_principalidade = ['ano_mes', 'cpf_cnpj', 'segmento', 'cod_cooperativa', 'cod_central',
       'cod_agencia', 'cod_carteira', 'status_associado', 'porte_padrao',
       'meses_desde_associacao', 'assoc_desde', 'nivel_risco', 'des_origem',
       'vlr_capital_social',  'pix_trans_30d', 'cad_pix',
       'sum_titulo_investimento', 'mobi_transacionou_30d', 'sum_previdencia',
       'sum_investimentos', 'total_valor_cartao_mercado_scr',
       'total_valor_cartao_sicredi', 'total_valor_outros_sicredi',  'sow_cartao',
       'possui_cartao_credito', 'debito_conta_ativo', 'possui_domicilio',
       'possui_cartao_debito', 'possui_folha_pagamento', 'possui_cobranca',
       'possui_adquirencia', 'transacao_app', 'flg_ativo', 'flg_novo_assoc',  'cad_pix_ativo',
       'ativou_open_finance', 'cash_in', 'cash_out', 'cash_total_fator',
       'cash_total', 'pontos_produtos_basicos',  'flag_principalidade',
       'pontos_principalidade', 'faixa_atingimento_meta',
       'percentual_atingimento',  'faixa_categoria', 'data_atualizacao',  'var_pontos',
       'queda_flag', 'soma_quedas']
colunas_principalidade

['ano_mes',
 'cpf_cnpj',
 'segmento',
 'cod_cooperativa',
 'cod_central',
 'cod_agencia',
 'cod_carteira',
 'status_associado',
 'porte_padrao',
 'meses_desde_associacao',
 'assoc_desde',
 'nivel_risco',
 'des_origem',
 'vlr_capital_social',
 'pix_trans_30d',
 'cad_pix',
 'sum_titulo_investimento',
 'mobi_transacionou_30d',
 'sum_previdencia',
 'sum_investimentos',
 'total_valor_cartao_mercado_scr',
 'total_valor_cartao_sicredi',
 'total_valor_outros_sicredi',
 'sow_cartao',
 'possui_cartao_credito',
 'debito_conta_ativo',
 'possui_domicilio',
 'possui_cartao_debito',
 'possui_folha_pagamento',
 'possui_cobranca',
 'possui_adquirencia',
 'transacao_app',
 'flg_ativo',
 'flg_novo_assoc',
 'cad_pix_ativo',
 'ativou_open_finance',
 'cash_in',
 'cash_out',
 'cash_total_fator',
 'cash_total',
 'pontos_produtos_basicos',
 'flag_principalidade',
 'pontos_principalidade',
 'faixa_atingimento_meta',
 'percentual_atingimento',
 'faixa_categoria',
 'data_atualizacao',
 'var_pontos',
 'queda_flag'

In [65]:
# Carregar parquet base gerado pela célula DuckDB e restringir ESTRITAMENTE às colunas definidas
import os, pandas as pd
ruta_base = rf"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\associados_queda_principalidade.parquet"
print('Lendo parquet base (fonte única principalidade):', ruta_base)
if not os.path.exists(ruta_base):
    raise FileNotFoundError('Parquet de principalidade não encontrado. Execute antes a célula DuckDB que o gera.')
_tmp = pd.read_parquet(ruta_base)
# Validar presença integral das colunas obrigatórias
faltantes = [c for c in colunas_principalidade if c not in _tmp.columns]
if faltantes:
    raise ValueError(f'Colunas obrigatórias ausentes no parquet: {faltantes}')
# Manter somente as colunas definidas (ordem garantida)
base_princ = _tmp[colunas_principalidade].copy()
print('Linhas:', len(base_princ), '| Colunas mantidas:', len(base_princ.columns))
base_princ.head(3)

Lendo parquet base (fonte única principalidade): C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\associados_queda_principalidade.parquet
Linhas: 3660 | Colunas mantidas: 50


,ano_mes,cpf_cnpj,segmento,cod_cooperativa,cod_central,cod_agencia,cod_carteira,status_associado,porte_padrao,meses_desde_associacao,assoc_desde,nivel_risco,des_origem,vlr_capital_social,pix_trans_30d,cad_pix,sum_titulo_investimento,mobi_transacionou_30d,sum_previdencia,sum_investimentos,total_valor_cartao_mercado_scr,total_valor_cartao_sicredi,total_valor_outros_sicredi,sow_cartao,possui_cartao_credito,debito_conta_ativo,possui_domicilio,possui_cartao_debito,possui_folha_pagamento,possui_cobranca,possui_adquirencia,transacao_app,flg_ativo,flg_novo_assoc,cad_pix_ativo,ativou_open_finance,cash_in,cash_out,cash_total_fator,cash_total,pontos_produtos_basicos,flag_principalidade,pontos_principalidade,faixa_atingimento_meta,percentual_atingimento,faixa_categoria,data_atualizacao,var_pontos,queda_flag,soma_quedas
0,202506,00355679965,PF,0730,7000,33,141,ATIVO,PF IV,32,202210,BAIXÍSSIMO,LEGADO,863.15,S,S,0.00,N,0.00,863.15,"74,831.14","4,876.56","14,168.99",0.07,1,1,0,0,0,0,0,1,1,0,1,0,0.00,"3,367.35",0.00,"3,367.35",16,0,30,20-80%,23.076923076923077,Emergente,2025-08-07,-2,1,3.00
1,202506,00403761921,PF,0730,7000,32,131,ATIVO,PF III,12,202406,BAIXÍSSIMO,LEGADO,277.81,N,S,"42,614.59",S,0.00,"42,892.40","12,633.76","4,249.86","1,726.91",0.34,1,1,0,0,0,0,0,1,1,0,0,0,0.00,"1,863.93",0.00,"1,863.93",12,0,89,80-100%,89.0,Promissor,2025-08-07,-3,1,3.00
2,202506,00427605938,PF,0730,7000,21,121,ATIVO,PF II,54,202012,BAIXO 2,LEGADO,"1,174.00",S,S,199.39,S,0.00,"1,373.39","3,294.48",0.00,"3,453.51",0.00,0,1,0,0,0,0,0,1,1,0,1,0,0.00,528.43,0.00,528.43,15,0,29,20-80%,36.25,Emergente,2025-08-07,-6,1,3.00


In [66]:
# (Removido salvamento antigo redundante) Placeholder para evitar erro.
print('Utilize a célula de montagem final para salvar os parquets.')

Utilize a célula de montagem final para salvar os parquets.


In [67]:
# Derivar produtos faltantes (executar após filtros e merges)
# Diferenciar PF vs PJ: assumir segmento ou comprimento do documento (11=PF, >11=PJ) se não houver coluna específica

prod_pf = {
    'Chave PIX ativa': 'cad_pix_ativo',
    'Débito automático': 'debito_conta_ativo',
    'Conta salário': 'possui_folha_pagamento',
    'Cartão de débito': 'possui_cartao_debito',
    'Cartão de crédito': 'possui_cartao_credito',
    'Canais digitais': 'transacao_app',
    'Open Finance (receptor)': 'ativou_open_finance'
}
prod_pj = {
    'Chave PIX ativa': 'cad_pix_ativo',
    'Débito automático': 'debito_conta_ativo',
    'Cartão de débito': 'possui_cartao_debito',
    'Cartão de crédito': 'possui_cartao_credito',
    'Canais digitais': 'transacao_app',
    'Open Finance (receptor)': 'ativou_open_finance',
    'Domicílio': 'possui_domicilio',
    'Credenciamento ativo': 'possui_adquirencia',
    'Cobrança': 'possui_cobranca',
    'Folha de Pagamento': 'possui_folha_pagamento'
}

# Métricas fluxo / SOW (iguais para ambos)
prod_cat2_metricas = {
    'Cash-in': 'cash_in',
    'Cash-out': 'cash_out',
    'Total movimentações': 'cash_total',
    'SOW Cartão': 'sow_cartao'
}

# Inferir tipo_pessoa
base_princ['tipo_pessoa'] = np.where(base_princ['cpf_cnpj'].str.len()==11, 'PF','PJ')

# Normalizar booleans
for col in set(list(prod_pf.values()) + list(prod_pj.values())):
    if col in base_princ.columns:
        base_princ[col] = base_princ[col].replace({'S':1,'N':0}).fillna(0).astype(float)

# Soma de chaves PIX fortes (se existisse colunas detalhadas, placeholder; usar cad_pix_ativo como 1 ou 0)
base_princ['qtde_chaves_pix_fortes'] = base_princ.get('cad_pix_ativo', pd.Series([0]*len(base_princ))).fillna(0)

# Produtos faltantes conforme tipo

def faltantes_prod(r):
    mapping = prod_pf if r['tipo_pessoa']=='PF' else prod_pj
    return [n for n,c in mapping.items() if c in base_princ.columns and r.get(c,0)==0]

base_princ['prod_cat1_faltantes'] = base_princ.apply(faltantes_prod, axis=1)

# Métricas categoria 2
base_princ['prod_cat2_metricas'] = base_princ.apply(
    lambda r: [f"{n}: {r.get(c)}" for n,c in prod_cat2_metricas.items() if c in base_princ.columns], axis=1)

base_princ.head(1)

,ano_mes,cpf_cnpj,segmento,cod_cooperativa,cod_central,cod_agencia,cod_carteira,status_associado,porte_padrao,meses_desde_associacao,assoc_desde,nivel_risco,des_origem,vlr_capital_social,pix_trans_30d,cad_pix,sum_titulo_investimento,mobi_transacionou_30d,sum_previdencia,sum_investimentos,total_valor_cartao_mercado_scr,total_valor_cartao_sicredi,total_valor_outros_sicredi,sow_cartao,possui_cartao_credito,debito_conta_ativo,possui_domicilio,possui_cartao_debito,possui_folha_pagamento,possui_cobranca,possui_adquirencia,transacao_app,flg_ativo,flg_novo_assoc,cad_pix_ativo,ativou_open_finance,cash_in,cash_out,cash_total_fator,cash_total,pontos_produtos_basicos,flag_principalidade,pontos_principalidade,faixa_atingimento_meta,percentual_atingimento,faixa_categoria,data_atualizacao,var_pontos,queda_flag,soma_quedas,tipo_pessoa,qtde_chaves_pix_fortes,prod_cat1_faltantes,prod_cat2_metricas
0,202506,00355679965,PF,0730,7000,33,141,ATIVO,PF IV,32,202210,BAIXÍSSIMO,LEGADO,863.15,S,S,0.00,N,0.00,863.15,"74,831.14","4,876.56","14,168.99",0.07,1.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1,0,1.00,0.00,0.00,"3,367.35",0.00,"3,367.35",16,0,30,20-80%,23.076923076923077,Emergente,2025-08-07,-2,1,3.00,PF,1.00,"[Conta salário, Cartão de débito, Open Finance (receptor)]","[Cash-in: 0.0, Cash-out: 3367.35009765625, Total movimentações: 3367.35009765625, SOW Cartão: 0.06516752392053604]"


In [68]:
base_princ

,ano_mes,cpf_cnpj,segmento,cod_cooperativa,cod_central,cod_agencia,cod_carteira,status_associado,porte_padrao,meses_desde_associacao,assoc_desde,nivel_risco,des_origem,vlr_capital_social,pix_trans_30d,cad_pix,sum_titulo_investimento,mobi_transacionou_30d,sum_previdencia,sum_investimentos,total_valor_cartao_mercado_scr,total_valor_cartao_sicredi,total_valor_outros_sicredi,sow_cartao,possui_cartao_credito,debito_conta_ativo,possui_domicilio,possui_cartao_debito,possui_folha_pagamento,possui_cobranca,possui_adquirencia,transacao_app,flg_ativo,flg_novo_assoc,cad_pix_ativo,ativou_open_finance,cash_in,cash_out,cash_total_fator,cash_total,pontos_produtos_basicos,flag_principalidade,pontos_principalidade,faixa_atingimento_meta,percentual_atingimento,faixa_categoria,data_atualizacao,var_pontos,queda_flag,soma_quedas,tipo_pessoa,qtde_chaves_pix_fortes,prod_cat1_faltantes,prod_cat2_metricas
0,202506,00355679965,PF,0730,7000,33,141,ATIVO,PF IV,32,202210,BAIXÍSSIMO,LEGADO,863.15,S,S,0.00,N,0.00,863.15,"74,831.14","4,876.56","14,168.99",0.07,1.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1,0,1.00,0.00,0.00,"3,367.35",0.00,"3,367.35",16,0,30,20-80%,23.076923076923077,Emergente,2025-08-07,-2,1,3.00,PF,1.00,"[Conta salário, Cartão de débito, Open Finance (receptor)]","[Cash-in: 0.0, Cash-out: 3367.35009765625, Total movimentações: 3367.35009765625, SOW Cartão: 0.06516752392053604]"
1,202506,00403761921,PF,0730,7000,32,131,ATIVO,PF III,12,202406,BAIXÍSSIMO,LEGADO,277.81,N,S,"42,614.59",S,0.00,"42,892.40","12,633.76","4,249.86","1,726.91",0.34,1.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1,0,0.00,0.00,0.00,"1,863.93",0.00,"1,863.93",12,0,89,80-100%,89.0,Promissor,2025-08-07,-3,1,3.00,PF,0.00,"[Chave PIX ativa, Conta salário, Cartão de débito, Open Finance (receptor)]","[Cash-in: 0.0, Cash-out: 1863.9300537109375, Total movimentações: 1863.9300537109375, SOW Cartão: 0.3363891541957855]"
2,202506,00427605938,PF,0730,7000,21,121,ATIVO,PF II,54,202012,BAIXO 2,LEGADO,"1,174.00",S,S,199.39,S,0.00,"1,373.39","3,294.48",0.00,"3,453.51",0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1,0,1.00,0.00,0.00,528.43,0.00,528.43,15,0,29,20-80%,36.25,Emergente,2025-08-07,-6,1,3.00,PF,1.00,"[Conta salário, Cartão de débito, Cartão de crédito, Open Finance (receptor)]","[Cash-in: 0.0, Cash-out: 528.4299926757812, Total movimentações: 528.4299926757812, SOW Cartão: 0.0]"
3,202506,00732074975,PF,0730,7000,27,141,ATIVO,PF IV,74,201904,ALTO 2,LEGADO,21.00,N,S,0.16,N,0.00,21.16,"13,832.09",0.00,"173,530.56",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,50,20-80%,38.46153846153847,Emergente,2025-08-07,-1,1,3.00,PF,0.00,"[Chave PIX ativa, Débito automático, Conta salário, Cartão de débito, Cartão de crédito, Canais digitais, Open Finance (receptor)]","[Cash-in: 0.0, Cash-out: 0.0, Total movimentações: 0.0, SOW Cartão: 0.0]"
4,202506,00772480907,PF,0730,7000,34,141,ATIVO,PF IV,51,202103,BAIXO 2,LEGADO,"1,352.03",N,N,0.00,N,0.00,"1,352.03","666,463.00","9,093.47",0.00,0.01,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,0,0.00,0.00,600.00,"2,097.66",0.00,"2,697.66",4,0,12,0-20%,9.230769230769232,Aspirante,2025-08-07,-13,1,3.00,PF,0.00,"[Chave PIX ativa, Débito automático, Conta salário, Cartão de débito, Canais digitais, Open Finance (receptor)]","[Cash-in: 600.0, Cash-out: 2097.659912109375, Total movimentações: 2697.659912109375, SOW Cartão: 0.01364437397569418]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655,202506,30626274842,PF,0730,7000,39,010201,ATIVO,PF II,19,202311,MÉDIO 1,DIGITAL,0.00,S,S,25.30,S,0.00,25.30,"2,640.10",48.85,981.90,0.02,1.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1,0,1.00,0.00,"2,266.80","3,035.38",0.00,"5,302.18",20,0,75,80-100%,93.75,Promissor,2025-08-07,-3,1,3.00,PF,1.00,"[Conta salário, Cartão de débito, Open Finance (receptor)]","[Cash-in: 2266.800048828125

In [ ]:
# Preparar y enriquecer la base principal: produtos, métricas, descripciones y salida
from datetime import datetime, timedelta
import os
import pandas as pd
import numpy as np

if 'base_princ' not in globals():
    raise RuntimeError('base_princ não carregada. Execute a célula de leitura do parquet base antes desta.')

# Mapeo de productos (clave descriptiva -> columna en base_princ)
prod_cols_map = {
    'Chave PIX ativa': 'cad_pix_ativo',
    'Débito automático': 'debito_conta_ativo',
    'Conta salário': 'possui_folha_pagamento',
    'Cartão de débito': 'possui_cartao_debito',
    'Cartão de crédito': 'possui_cartao_credito',
    'Canais digitais': 'transacao_app',
    'Open Finance (receptor)': 'ativou_open_finance',
    'Domicílio': 'possui_domicilio',
    'Credenciamento ativo': 'possui_adquirencia',
    'Cobrança': 'possui_cobranca'
}

# Normalizar columnas de productos (S/N -> 1/0) si existen
for col in set(prod_cols_map.values()):
    if col in base_princ.columns:
        base_princ[col] = base_princ[col].replace({'S': 1, 'N': 0}).fillna(0).astype(float)

# Inferir tipo_pessoa si no existe
if 'tipo_pessoa' not in base_princ.columns and 'cpf_cnpj' in base_princ.columns:
    base_princ['tipo_pessoa'] = np.where(base_princ['cpf_cnpj'].astype(str).str.len() == 11, 'PF', 'PJ')

# Función para listar productos faltantes por cliente
def _falt(r):
    tipo = r.get('tipo_pessoa', 'PF')
    # Para PF excluimos productos que no aplican típicamente
    excluidos_pf = {'Domicílio', 'Credenciamento ativo', 'Cobrança'}
    activos = {k: v for k, v in prod_cols_map.items() if not (tipo == 'PF' and k in excluidos_pf)}
    faltantes = []
    for nome, col in activos.items():
        if col in r.index and pd.notnull(r.get(col)):
            try:
                if float(r.get(col)) == 0:
                    faltantes.append(nome)
            except Exception:
                # si no es numérico, considerarlo faltante si es falsy
                if not r.get(col):
                    faltantes.append(nome)
        else:
            # si la columna no existe en el registro, lo ignoramos
            continue
    return faltantes

base_princ['prod_cat1_faltantes'] = base_princ.apply(_falt, axis=1)

# Métricas de flujo / SOW: etiquetas -> columnas esperadas en la base
prod_cat2_metricas = {
    'Cash-in': 'cash_in',
    'Cash-out': 'cash_out',
    'Cash-total movimentações': 'cash_total',
    'SOW Total Cartão': 'sow_cartao'
}

def _fmt_val(v):
    try:
        return f"{float(v):.2f}"
    except Exception:
        return '' if pd.isna(v) else str(v)

def _monta_metricas(r):
    itens = []
    for nome, col in prod_cat2_metricas.items():
        if col in r.index and pd.notnull(r.get(col)):
            itens.append(f"{nome}: {_fmt_val(r.get(col))}")
    return itens

base_princ['prod_cat2_metricas'] = base_princ.apply(_monta_metricas, axis=1)

# Título para o chamado
base_princ['titulo'] = base_princ.apply(lambda r: f"Oferta Principalidade - {r.get('cpf_cnpj', '')}", axis=1)

# Determinar columna ISA atual
col_isa_atual = None
if 'isa_atual_col' in globals() and isinstance(isa_atual_col, str) and isa_atual_col in base_princ.columns:
    col_isa_atual = isa_atual_col
elif 'isa_media' in base_princ.columns:
    col_isa_atual = 'isa_media'

def _fmt_isa_media3m(v):
    try:
        return f"{float(v):.2f}"
    except Exception:
        return ''

if col_isa_atual:
    def _bloco_isa(r):
        atual = r.get(col_isa_atual, '')
        media3m = _fmt_isa_media3m(r.get('isa_media_3m_calc')) if 'isa_media_3m_calc' in r.index else ''
        partes = []
        if pd.notnull(atual) and atual != '':
            partes.append(f"ISA atual: {atual}")
        if media3m:
            partes.append(f"ISA média (3M): {media3m}")
        return ' | '.join(partes)
    base_princ['bloco_isa'] = base_princ.apply(_bloco_isa, axis=1)
else:
    base_princ['bloco_isa'] = ''

# Bloques de productos e indicadores
base_princ['bloco_cat1'] = base_princ['prod_cat1_faltantes'].apply(lambda lst: 'Produtos básicos completos' if not lst else 'Produtos a ofertar: ' + ', '.join(lst))
base_princ['bloco_cat2'] = base_princ['prod_cat2_metricas'].apply(lambda lst: ' / '.join(lst) if lst else '')

# Descripción combinada
def _descricao(r):
    parts = [f"Segmento: {r.get('segmento','')}", f"MC | Total: {r.get('cash_total','')}"]
    if r.get('bloco_isa'): parts.append(r.get('bloco_isa'))
    if r.get('bloco_cat1'): parts.append(r.get('bloco_cat1'))
    if r.get('bloco_cat2'): parts.append(r.get('bloco_cat2'))
    return ' | '.join([p for p in parts if p])

base_princ['descricao'] = base_princ.apply(_descricao, axis=1)

# Categoria do chamado
base_princ['categoriaChamadoId'] = base_princ['prod_cat1_faltantes'].apply(lambda l: 1 if len(l) > 0 else 2)
nome_map = {1: 'Acesso', 2: 'Fluxo de Caixa'}
base_princ['nomeCategoria'] = base_princ['categoriaChamadoId'].map(nome_map)

desc_map = {1: 'Ofertar Básicos', 2: 'Ofertar Serviços de Movimentações'}
base_princ['descricaoCategoria'] = base_princ['categoriaChamadoId'].map(desc_map)

# Ordenar por pontos de principalidade se existir
if 'pontos_principalidade' in base_princ.columns:
    base_princ = base_princ.sort_values('pontos_principalidade').reset_index(drop=True)
else:
    base_princ = base_princ.reset_index(drop=True)

# Distribuir datas de necessidade ciclando por 4 semanas (0,7,14,21)
qtd = len(base_princ)
ini = datetime.today().date()
semanas = [0, 7, 14, 21]
if qtd > 0:
    base_princ['dataNecessidade'] = base_princ.index.map(lambda i: (ini + timedelta(days=semanas[i % len(semanas)])).isoformat() + 'T08:00:00.000Z')
else:
    base_princ['dataNecessidade'] = []

# Montar campo categoriaChamado (estrutura esperada)
def _monta_categoria(r):
    return {
        'categoriaChamadoId': int(r['categoriaChamadoId']),
        'nomeCategoria': r.get('nomeCategoria',''),
        'descricaoCategoria': r.get('descricaoCategoria',''),
        'dataCriacao': datetime.today().date().isoformat(),
        'departamentoId': int(r.get('cod_agencia', 0))
    }

base_princ['categoriaChamado'] = base_princ.apply(_monta_categoria, axis=1)

# Preparar base_final renombrando columnas esperadas
base_final = base_princ.rename(columns={'cod_agencia': 'numeroAgencia', 'nome_agencia': 'nomeDepartamento'})

# Responsable / solicitante
usuario_default = globals().get('user', '') or globals().get('USER', '')
base_final['usuarioResponsavel'] = base_final.get('gestor', usuario_default)
base_final['usuarioSolicitante'] = base_final['usuarioResponsavel']
base_final['statusChamado'] = 'RASCUNHO'
base_final['departamentoId'] = base_final['numeroAgencia']

# Selección columnas de salida
cols_saida = ['titulo', 'usuarioResponsavel', 'usuarioSolicitante', 'numeroAgencia',
             'descricao', 'statusChamado', 'departamentoId', 'nomeDepartamento',
             'dataNecessidade', 'categoriaChamado']

resultado_chamados = base_final.loc[:, [c for c in cols_saida if c in base_final.columns]].copy()

# Salvar arquivos parquet
saida_dir = RUTAS.get('saida_dir_giro') if isinstance(RUTAS, dict) else None
if saida_dir:
    parquet_completo = os.path.join(saida_dir, 'giro_completo_principalidade.parquet')
    parquet_chamados = os.path.join(saida_dir, 'giro_chamados_principalidade.parquet')
    try:
        base_princ.to_parquet(parquet_completo, index=False)
        resultado_chamados.to_parquet(parquet_chamados, index=False)
        print({'completo': parquet_completo, 'chamados': parquet_chamados, 'linhas_chamados': len(resultado_chamados)})
    except Exception as e:
        print('Erro ao salvar parquet:', e)
else:
    print('RUTAS["saida_dir_giro"] não encontrada; não foi possível salvar os arquivos.')

resultado_chamados.to_parquet(rf"")


{'completo': 'C:\\Users\\carola_luco\\Sicredi\\TimeBI_0730 - Documentos\\01_Rotineiros\\33_GiroCarteira\\giro_de_carteira\\giro_completo_principalidade.parquet', 'chamados': 'C:\\Users\\carola_luco\\Sicredi\\TimeBI_0730 - Documentos\\01_Rotineiros\\33_GiroCarteira\\giro_de_carteira\\giro_chamados_principalidade.parquet', 'linhas_chamados': 3660}


,titulo,usuarioResponsavel,usuarioSolicitante,numeroAgencia,descricao,statusChamado,departamentoId,dataNecessidade,categoriaChamado
0,Oferta Principalidade - 06941229985,carola_luco,carola_luco,08,"Segmento: PF | MC | Total: 0.0 | Produtos a ofertar: Chave PIX ativa, Débito automático, Conta salário, Cartão de débito, Cartão de crédito, Canais digitais, Open Finance (receptor) | Cash-in: 0.00 / Cash-out: 0.00 / Cash-total movimentações: 0.00 / SOW Total Cartão: 0.00",RASCUNHO,08,2025-08-20T08:00:00.000Z,"{'categoriaChamadoId': 1, 'nomeCategoria': 'Acesso', 'descricaoCategoria': 'Ofertar Básicos', 'dataCriacao': '2025-08-20', 'departamentoId': 8}"


In [71]:
# Aplicar filtros e preparar base principal (usa base_princ já carregada do parquet associados_queda_principalidade)
if 'base_princ' not in globals():
    raise RuntimeError('base_princ não carregada. Execute a célula de leitura do parquet base antes desta.')

if base_princ.empty:
    raise ValueError('base_princ está vazia. Verifique o parquet base.')

# Normalizar chave
def normaliza_doc(valor):
    if pd.isna(valor):
        return valor
    v = re.sub(r'\D','', str(valor))
    return v.lstrip('0')

if 'cpf_cnpj' not in base_princ.columns:
    possiveis_chaves = [c for c in base_princ.columns if 'cpf' in c.lower() or 'cnpj' in c.lower() or 'doc' in c.lower()]
    if possiveis_chaves:
        base_princ.rename(columns={possiveis_chaves[0]:'cpf_cnpj'}, inplace=True)
if 'cpf_cnpj' not in base_princ.columns:
    raise ValueError('Coluna cpf_cnpj ausente na base principalidade mês.')
base_princ['cpf_cnpj'] = base_princ['cpf_cnpj'].map(normaliza_doc)

# Inadimplentes
path_inad = RUTAS['inadimplentes']
if os.path.exists(path_inad):
    inad = pd.read_parquet(path_inad)
    for alt in ['cpf','cnpj','documento','doc','num_cpf_cnpj']:
        if alt in inad.columns and 'cpf_cnpj' not in inad.columns:
            inad.rename(columns={alt:'cpf_cnpj'}, inplace=True)
    if 'cpf_cnpj' not in inad.columns:
        inad['cpf_cnpj'] = inad.iloc[:,0]
    inad['cpf_cnpj'] = inad['cpf_cnpj'].map(normaliza_doc)
    inad = inad[['cpf_cnpj']].drop_duplicates()
    inad['flag_inadimplente'] = 1
    base_princ = base_princ.merge(inad, on='cpf_cnpj', how='left')
else:
    base_princ['flag_inadimplente'] = np.nan

# Risco BBM (normalizar caixa / acentuação mínima)
if 'nivel_risco' in base_princ.columns:
    base_princ['nivel_risco_norm'] = base_princ['nivel_risco'].astype(str).str.upper().str.strip()
    riscos_ok = [r.upper() for r in PARAMS['filtros_risco_bbm']]
    base_princ = base_princ[base_princ['nivel_risco_norm'].isin(riscos_ok)]
else:
    print('Aviso: coluna nivel_risco ausente.')

# Sem inadimplência
base_princ = base_princ[(base_princ['flag_inadimplente'].isna()) | (base_princ['flag_inadimplente']!=1)]

# Merge dashboard (inclui ult_movimento)
if 'assoc_dash' in globals():
    assoc_dash['cpf_cnpj'] = assoc_dash['cpf_cnpj'].map(normaliza_doc)
    base_princ = base_princ.merge(assoc_dash, on='cpf_cnpj', how='left')
else:
    print('Dashboard não carregado: execute a célula de carga do dashboard/ISA antes desta.')

# Filtro movimento 20-45 dias usando ult_movimento
if 'ult_movimento' in base_princ.columns:
    hoje = pd.Timestamp.today().normalize()
    base_princ['ult_movimento_dt'] = pd.to_datetime(base_princ['ult_movimento'], errors='coerce')
    base_princ['dias_sem_mov'] = (hoje - base_princ['ult_movimento_dt']).dt.days
    lim_inf, lim_sup = PARAMS['dias_sem_movimentacao']
    base_princ = base_princ[(base_princ['dias_sem_mov']>=lim_inf) & (base_princ['dias_sem_mov']<=lim_sup)]
else:
    print('Coluna ult_movimento ausente no dashboard para filtro de movimento.')

# Merge ISA se disponível
if 'isa_sel' in globals() and isinstance(isa_sel, pd.DataFrame) and not isa_sel.empty and 'cpf_cnpj' in isa_sel.columns:
    isa_sel['cpf_cnpj'] = isa_sel['cpf_cnpj'].map(normaliza_doc)
    if 'gestor' in base_princ.columns and 'gestor' in isa_sel.columns:
        base_princ = base_princ.merge(isa_sel, on=['cpf_cnpj','gestor'], how='left')
    else:
        base_princ = base_princ.merge(isa_sel, on='cpf_cnpj', how='left')
else:
    base_princ['isa_media_3m_calc'] = np.nan

# Corte ISA >= 2 (média atual ou média 3M). Remove quem está fora.
if 'isa_media' in base_princ.columns:
    base_princ = base_princ[(base_princ['isa_media'].fillna(0) >= 2) | (base_princ.get('isa_media_3m_calc', pd.Series(dtype=float)).fillna(0) >= 2)]

print('Base após filtros (inclui corte ISA>=2):', base_princ.shape)
base_princ.head(2)

Dashboard não carregado: execute a célula de carga do dashboard/ISA antes desta.
Coluna ult_movimento ausente no dashboard para filtro de movimento.
Base após filtros (inclui corte ISA>=2): (2412, 67)


,ano_mes,cpf_cnpj,segmento,cod_cooperativa,cod_central,cod_agencia,cod_carteira,status_associado,porte_padrao,meses_desde_associacao,assoc_desde,nivel_risco,des_origem,vlr_capital_social,pix_trans_30d,cad_pix,sum_titulo_investimento,mobi_transacionou_30d,sum_previdencia,sum_investimentos,total_valor_cartao_mercado_scr,total_valor_cartao_sicredi,total_valor_outros_sicredi,sow_cartao,possui_cartao_credito,debito_conta_ativo,possui_domicilio,possui_cartao_debito,possui_folha_pagamento,possui_cobranca,possui_adquirencia,transacao_app,flg_ativo,flg_novo_assoc,cad_pix_ativo,ativou_open_finance,cash_in,cash_out,cash_total_fator,cash_total,pontos_produtos_basicos,flag_principalidade,pontos_principalidade,faixa_atingimento_meta,percentual_atingimento,faixa_categoria,data_atualizacao,var_pontos,queda_flag,soma_quedas,tipo_pessoa,qtde_chaves_pix_fortes,prod_cat1_faltantes,prod_cat2_metricas,titulo,bloco_isa,bloco_cat1,bloco_cat2,descricao,categoriaChamadoId,nomeCategoria,descricaoCategoria,dataNecessidade,categoriaChamado,flag_inadimplente,nivel_risco_norm,isa_media_3m_calc
0,202506,6941229985,PF,0730,7000,08,019911,ATIVO,PF I,5,202501,BAIXO 2,DIGITAL,0.00,N,S,0.00,N,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0-20%,0.0,Aspirante,2025-08-07,-2,1,3.00,PF,0.00,"[Chave PIX ativa, Débito automático, Conta salário, Cartão de débito, Cartão de crédito, Canais digitais, Open Finance (receptor)]","[Cash-in: 0.00, Cash-out: 0.00, Cash-total movimentações: 0.00, SOW Total Cartão: 0.00]",Oferta Principalidade - 06941229985,,"Produtos a ofertar: Chave PIX ativa, Débito automático, Conta salário, Cartão de débito, Cartão de crédito, Canais digitais, Open Finance (receptor)",Cash-in: 0.00 / Cash-out: 0.00 / Cash-total movimentações: 0.00 / SOW Total Cartão: 0.00,"Segmento: PF | MC | Total: 0.0 | Produtos a ofertar: Chave PIX ativa, Débito automático, Conta salário, Cartão de débito, Cartão de crédito, Canais digitais, Open Finance (receptor) | Cash-in: 0.00 / Cash-out: 0.00 / Cash-total movimentações: 0.00 / SOW Total Cartão: 0.00",1,Acesso,Ofertar Básicos,2025-08-20T08:00:00.000Z,"{'categoriaChamadoId': 1, 'nomeCategoria': 'Acesso', 'descricaoCategoria': 'Ofertar Básicos', 'dataCriacao': '2025-08-20', 'departamentoId': 8}",NaN,BAIXO 2,NaN
3,202506,43229071808,PF,0730,7000,22,010201,ATIVO,PF II,38,202204,MÉDIO 1,DIGITAL,0.00,N,S,0.00,N,0.00,0.00,"2,736.52",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0-20%,0.0,Aspirante,2025-08-07,-10,1,3.00,PF,0.00,"[Chave PIX ativa, Débito automático, Conta salário, Cartão de débito, Cartão de crédito, Canais digitais, Open Finance (receptor)]","[Cash-in: 0.00, Cash-out: 0.00, Cash-total movimentações: 0.00, SOW Total Cartão: 0.00]",Oferta Principalidade - 43229071808,,"Produtos a ofertar: Chave PIX ativa, Débito automático, Conta salário, Cartão de débito, Cartão de crédito, Canais digitais, Open Finance (receptor)",Cash-in: 0.00 / Cash-out: 0.00 / Cash-total movimentações: 0.00 / SOW Total Cartão: 0.00,"Segmento: PF | MC | Total: 0.0 | Produtos a ofertar: Chave PIX ativa, Débito automático, Conta salário, Cartão de débito, Cartão de crédito, Canais digitais, Open Finance (receptor) | Cash-in: 0.00 / Cash-out: 0.00 / Cash-total movimentações: 0.00 / SOW Total Cartão: 0.00",1,Acesso,Ofertar Básicos,2025-09-10T08:00:00.000Z,"{'categoriaChamadoId': 1, 'nomeCategoria': 'Acesso', 'descricaoCategoria': 'Ofertar Básicos', 'dataCriacao': '2025-08-20', 'departamentoId': 22}",NaN,MÉDIO 1,NaN


In [ ]:
# Carregar dashboard associados e ISA histórico
import pandas as pd, glob, re, numpy as np

path_dashboard = rf"C:\Users\{user}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\associados_totais_tratados_dashboard.parquet"
print('Carregando dashboard:', path_dashboard)
# Incluir ult_movimento para filtro correto
assoc_dash = pd.read_parquet(path_dashboard, columns=['cpf_cnpj','nome_agencia','mc_total_1','gestor','ult_movimento'])
print('Assoc dash linhas:', len(assoc_dash))

# ISA histórico: pegar parquet mais recente na pasta config['pasta_isa_historico']
isa_dir = config['pasta_isa_historico']
isa_files = sorted(glob.glob(os.path.join(isa_dir, 'isa_historico_analise_*.parquet')))
if not isa_files:
    print('Aviso: Nenhum arquivo ISA encontrado em', isa_dir, '- prosseguindo sem ISA.')
    isa_sel = pd.DataFrame(columns=['cpf_cnpj','gestor','isa_status','isa_media','isa_media_3m_calc'])
    isa_atual_col = None
else:
    ultimo_isa = isa_files[-1]
    print('Usando ISA histórico:', ultimo_isa)
    isa = pd.read_parquet(ultimo_isa)
    # Detectar colunas mensais isa_YYYYMM e pegar as 4 últimas para calcular média 3M (usando 3 mais recentes excluindo atual para média?)
    col_isa_mes = [c for c in isa.columns if re.match(r'isa_\d{6}$', c)]
    col_isa_mes_sorted = sorted(col_isa_mes)
    ultimas4 = col_isa_mes_sorted[-4:]
    print('Últimas colunas ISA detectadas:', ultimas4)

    # Separar atual e anteriores (assumir última é atual)
    if len(ultimas4) >= 4:
        isa_atual_col = ultimas4[-1]
        tres_anteriores = ultimas4[-4:-1]
    else:
        isa_atual_col = ultimas4[-1]
        tres_anteriores = ultimas4[:-1]
    print('ISA atual:', isa_atual_col, '| 3 anteriores:', tres_anteriores)

    base_cols = ['cpf_cnpj','gestor','isa_status','isa_media'] + tres_anteriores + [isa_atual_col]
    base_cols = [c for c in base_cols if c in isa.columns]
    isa_sel = isa[base_cols].copy()
    if tres_anteriores:
        cols_exist = [c for c in tres_anteriores if c in isa_sel.columns]
        isa_sel['isa_media_3m_calc'] = isa_sel[cols_exist].mean(axis=1)
    else:
        isa_sel['isa_media_3m_calc'] = np.nan
print('Cols ISA finais:', isa_sel.columns.tolist())
isa_sel.head()

In [ ]:
# Debug: inspecionar colunas e tipos do parquet de associados (dashboard)
print('Colunas disponíveis no dashboard:', assoc_dash.columns.tolist())
print('\nTipos:')
print(assoc_dash.dtypes)
print('\nAmostra de ult_movimento e gestor:')
print(assoc_dash[['cpf_cnpj','gestor','ult_movimento']].head(5))

{  
  "titulo": "string",    
  "usuarioResponsavel": "string", ----> GESTOR
  "usuarioSolicitante": "string", -----> 
  "numeroAgencia": 0, -----> cod_agencia
  "descricao": "string",  --------> descricao texto com detalhes e indices do associado + o texto da oferta de produtos a entregar
  "statusChamado": "RASCUNHO", 
  "departamentoId": 0,
  "nomeDepartamento": "string", --------->  _nom_agencia
  "dataNecessidade": "2025-08-14T11:45:45.991Z",
  "categoriaChamado": {
    "categoriaChamadoId": 0,  ---- > 1 ou 2
    "nomeCategoria": "string", ------> 
    "descricaoCategoria": "string", 
    "dataCriacao": "2025-08-14",
    "departamentoId": 0}
}

Titulo () + nome associado + cpf 

Descrição:
PF OU PJ
ISA atual 
ISA 3M
MC TOTAL ATUAL
MC 6M
INDICES PRINCIPALIDADE

OFERTA : TEXTO conformado por  categoria + descriçao categoria + produto  1 texto : 

  Oferta Acesso  ----->  Categoria 1 
(descriçao categoria) Ofertar Produtos/Serviços Básicos : {} ou [] {add. Produto}-------> Titulo + nome associado + cpf  
. Chave PIX ativa 
· Débito automático
· Cartão de débito
· Cartão de crédito
. Canais digitais
. Credenciamento ativo
· Cobrança
. Folha de Pagamento
. Domicílio
. Open Finance (receptor)

 Oferta Fluxo de Caixa   ----->  Categoria 2 
if % produto in range 
(descriçao categoria) Ofertar Movimentações : [] ou {} {add. Crédito &| cartão}-------> Titulo + nome associado + cpf
. Somatório do total de movimentações:
- Receber (Cash-in)
- Pagar (Cash-out)

Crédito - cartão
. SOW (Share of Wallet)
referente à cartão

Credito - outros
SOW (Share of Wallet)
referente à créditos




In [ ]:
# Gerar parquets: completo (dados filtrados) e card Kanban
import json

if 'base_princ' not in globals() or 'resultado_chamados' not in globals():
    raise RuntimeError('Execute as células anteriores antes de gerar os parquets finais.')

# 1. Parquet completo já salvo anteriormente, mas reforçamos aqui se desejar atualizar
parquet_completo = os.path.join(RUTAS['saida_dir_giro'], 'giro_completo_principalidade.parquet')
base_princ.to_parquet(parquet_completo, index=False)

# 2. Parquet apenas com campos necessários para o card (Kanban)
cols_card = [
    'titulo',
    'descricao',
    'categoriaChamado',
    'usuarioResponsavel',
    'usuarioSolicitante',
    'dataNecessidade',
    'statusChamado',
    'numeroAgencia',
    'nomeDepartamento',
    'departamentoId'
]
card_df = resultado_chamados[cols_card].copy()

# Opcional: serializar categoriaChamado em JSON para consumo externo se necessário
card_df['categoriaChamado_json'] = card_df['categoriaChamado'].apply(lambda x: json.dumps(x, ensure_ascii=False))

parquet_card = os.path.join(RUTAS['saida_dir_giro'], 'giro_cards_kanban.parquet')
card_df.to_parquet(parquet_card, index=False)

print({'parquet_completo': parquet_completo, 'parquet_card': parquet_card, 'linhas_card': len(card_df)})
card_df.head(1)

In [ ]:
# PIPELINE FINAL UNIFICADA
import json
from datetime import datetime, timedelta

if 'base_princ' not in globals():
    raise RuntimeError('base_princ ausente. Execute leitura base.')

# Garantir campos essenciais derivados
if 'prod_cat1_faltantes' not in base_princ.columns:
    raise RuntimeError('Execute a célula de derivação de produtos antes da pipeline final.')

# Família pontuação por grupos (Acesso = produtos básicos, Fluxo = cash, SOW = sow_cartao)
base_princ['score_acesso'] = base_princ.get('pontos_produtos_basicos', np.nan)
base_princ['score_fluxo'] = base_princ.get('cash_total_fator', np.nan)
base_princ['score_sow'] = base_princ.get('sow_cartao', np.nan)

# Classificação novo associado (flg_novo_assoc ou meses_desde_associacao)
base_princ['class_assoc'] = np.where(
    (base_princ.get('flg_novo_assoc',0)==1) | (base_princ.get('meses_desde_associacao',999)<=6),
    'Novo', 'Base Giro'
)

# Blocos texto adicionais
base_princ['bloco_principalidade'] = base_princ.apply(lambda r: f"Faixa: {r.get('faixa_categoria')} | Var pts: {r.get('var_pontos')} | Queda flag: {r.get('queda_flag')} | Soma quedas: {r.get('soma_quedas')}", axis=1)
base_princ['bloco_familias'] = base_princ.apply(lambda r: f"Pontuações -> Acesso: {r.get('score_acesso')} | Fluxo: {r.get('score_fluxo')} | SOW Cartão: {r.get('score_sow')}", axis=1)
base_princ['bloco_tipo_pix'] = base_princ.apply(lambda r: f"Tipo Pessoa: {r.get('tipo_pessoa')} | Chaves PIX fortes: {r.get('qtde_chaves_pix_fortes')}", axis=1)
base_princ['bloco_assoc'] = base_princ['class_assoc'].apply(lambda v: f"Classificação Associação: {v}")

# Bloco ISA (recalcular se necessário)
base_princ['bloco_isa'] = base_princ.apply(lambda r: (
    f"ISA atual: {r.get('isa_media')} | ISA média 3M: {r.get('isa_media_3m_calc'):.2f}" if pd.notnull(r.get('isa_media_3m_calc')) else (
        f"ISA atual: {r.get('isa_media')}" if pd.notnull(r.get('isa_media')) else ''
    )
), axis=1)

base_princ['bloco_prod_basicos'] = base_princ['prod_cat1_faltantes'].apply(lambda lst: ('Produtos básicos a ofertar: ' + ', '.join(lst)) if lst else 'Produtos básicos completos')
base_princ['bloco_fluxo'] = base_princ['prod_cat2_metricas'].apply(lambda m: 'Indicadores fluxo / SOW: ' + '; '.join(m))

# Título e descrição (incluindo novos blocos)
base_princ['titulo'] = base_princ.apply(lambda r: f"Oferta Principalidade - {r.get('cpf_cnpj')}", axis=1)
base_princ['descricao'] = base_princ.apply(lambda r: ' | '.join([
    f"Segmento: {r.get('segmento')}",
    r.get('bloco_tipo_pix',''),
    r.get('bloco_assoc',''),
    r.get('bloco_isa',''),
    r.get('bloco_principalidade',''),
    r.get('bloco_familias',''),
    f"MC Total Atual: {r.get('cash_total')}",
    r.get('bloco_prod_basicos',''),
    r.get('bloco_fluxo','')
]), axis=1)

# Categoria (1 se há produto faltante, senão 2)
base_princ['categoriaChamadoId'] = base_princ['prod_cat1_faltantes'].apply(lambda l: 1 if len(l)>0 else 2)
base_princ['nomeCategoria'] = base_princ['categoriaChamadoId'].map({1:'Acesso',2:'Fluxo de Caixa'})
base_princ['descricaoCategoria'] = base_princ['nomeCategoria'].map({'Acesso':'Ofertar Produtos e Serviços Básicos','Fluxo de Caixa':'Ofertar Movimentações e Uso de Crédito'})

# Distribuir datas
base_princ = base_princ.sort_values('pontos_principalidade').reset_index(drop=True)
qtd = len(base_princ)
ini_mes = datetime.today().replace(day=1).date()
semanas = [0,7,14,21]
base_princ['dataNecessidade'] = base_princ.index.map(lambda i: (ini_mes + timedelta(days=semanas[(i * 4 // (qtd if qtd>0 else 1)) % 4])).isoformat()+'T08:00:00.000Z')

base_princ['categoriaChamado'] = base_princ.apply(lambda r: {
    'categoriaChamadoId': int(r['categoriaChamadoId']),
    'nomeCategoria': r['nomeCategoria'],
    'descricaoCategoria': r['descricaoCategoria'],
    'dataCriacao': datetime.today().date().isoformat(),
    'departamentoId': int(r.get('cod_agencia',0))
}, axis=1)

base_princ['usuarioResponsavel'] = base_princ.get('gestor') if 'gestor' in base_princ.columns else ''
base_princ['usuarioSolicitante'] = USER
base_princ['statusChamado'] = 'RASCUNHO'
base_princ['numeroAgencia'] = base_princ['cod_agencia']
base_princ['nomeDepartamento'] = base_princ.get('nome_agencia') if 'nome_agencia' in base_princ.columns else ''
base_princ['departamentoId'] = base_princ['numeroAgencia']

cols_kanban = ['titulo','usuarioResponsavel','usuarioSolicitante','numeroAgencia','descricao','statusChamado','departamentoId','nomeDepartamento','dataNecessidade','categoriaChamado']
resultado_chamados = base_princ[cols_kanban].copy()

PARQUET_COMPLETO = os.path.join(RUTAS['saida_dir_giro'], 'giro_completo_principalidade.parquet')
PARQUET_KANBAN = os.path.join(RUTAS['saida_dir_giro'], 'giro_cards_kanban.parquet')
base_princ.to_parquet(PARQUET_COMPLETO, index=False)
resultado_chamados.to_parquet(PARQUET_KANBAN, index=False)
print({'completo': PARQUET_COMPLETO, 'kanban': PARQUET_KANBAN, 'linhas': len(resultado_chamados)})
resultado_chamados.head(3)

In [ ]:
# PIPELINE FINAL UNIFICADA
import json
from datetime import datetime, timedelta

if 'base_princ' not in globals():
    raise RuntimeError('base_princ ausente. Execute leitura base.')

# Garantir que produtos faltantes e métricas já existem, caso célula anterior não tenha sido executada
if 'prod_cat1_faltantes' not in base_princ.columns:
    prod_cat1_cols = {
        'Chave PIX ativa': 'cad_pix_ativo',
        'Débito automático': 'debito_conta_ativo',
        'Cartão de débito': 'possui_cartao_debito',
        'Cartão de crédito': 'possui_cartao_credito',
        'Canais digitais': 'transacao_app',
        'Credenciamento ativo': 'possui_adquirencia',
        'Cobrança': 'possui_cobranca',
        'Folha de Pagamento': 'possui_folha_pagamento',
        'Domicílio': 'possui_domicilio',
        'Open Finance (receptor)': 'ativou_open_finance'
    }
    for c in prod_cat1_cols.values():
        if c in base_princ.columns:
            base_princ[c] = base_princ[c].replace({'S':1,'N':0}).fillna(0).astype(float)
    base_princ['prod_cat1_faltantes'] = base_princ.apply(lambda r: [n for n,c in prod_cat1_cols.items() if c in base_princ.columns and r.get(c)==0], axis=1)

if 'prod_cat2_metricas' not in base_princ.columns:
    prod_cat2_metricas = {
        'Cash-in': 'cash_in',
        'Cash-out': 'cash_out',
        'Total movimentações': 'cash_total',
        'SOW Cartão': 'sow_cartao'
    }
    base_princ['prod_cat2_metricas'] = base_princ.apply(lambda r: [f"{n}: {r.get(c)}" for n,c in prod_cat2_metricas.items() if c in base_princ.columns], axis=1)

# Blocos texto
base_princ['bloco_isa'] = base_princ.apply(lambda r: (
    f"ISA atual: {r.get('isa_media')} | ISA média 3M: {r.get('isa_media_3m_calc'):.2f}" if pd.notnull(r.get('isa_media_3m_calc')) else (
        f"ISA atual: {r.get('isa_media')}" if pd.notnull(r.get('isa_media')) else ''
    )
), axis=1)
base_princ['bloco_prod_basicos'] = base_princ['prod_cat1_faltantes'].apply(lambda lst: ('Produtos básicos a ofertar: ' + ', '.join(lst)) if lst else 'Produtos básicos completos')
base_princ['bloco_fluxo'] = base_princ['prod_cat2_metricas'].apply(lambda m: 'Indicadores fluxo / SOW: ' + '; '.join(m))

# Título e descrição
base_princ['titulo'] = base_princ.apply(lambda r: f"Oferta Principalidade - {r.get('cpf_cnpj')}", axis=1)
base_princ['descricao'] = base_princ.apply(lambda r: ' | '.join([
    f"Segmento: {r.get('segmento')}",
    r.get('bloco_isa',''),
    f"MC Total Atual: {r.get('cash_total')}",
    r.get('bloco_prod_basicos',''),
    r.get('bloco_fluxo','')
]), axis=1)

# Categoria (1 se há produto faltante, senão 2)
base_princ['categoriaChamadoId'] = base_princ['prod_cat1_faltantes'].apply(lambda l: 1 if len(l)>0 else 2)
base_princ['nomeCategoria'] = base_princ['categoriaChamadoId'].map({1:'Acesso',2:'Fluxo de Caixa'})
base_princ['descricaoCategoria'] = base_princ['nomeCategoria'].map({'Acesso':'Ofertar Produtos e Serviços Básicos','Fluxo de Caixa':'Ofertar Movimentações e Uso de Crédito'})

# Distribuir datas em 4 grupos
base_princ = base_princ.sort_values('pontos_principalidade').reset_index(drop=True)
qtd = len(base_princ)
ini_mes = datetime.today().replace(day=1).date()
semanas = [0,7,14,21]
base_princ['dataNecessidade'] = base_princ.index.map(lambda i: (ini_mes + timedelta(days=semanas[(i * 4 // (qtd if qtd>0 else 1)) % 4])).isoformat()+'T08:00:00.000Z')

# Objeto categoriaChamado
base_princ['categoriaChamado'] = base_princ.apply(lambda r: {
    'categoriaChamadoId': int(r['categoriaChamadoId']),
    'nomeCategoria': r['nomeCategoria'],
    'descricaoCategoria': r['descricaoCategoria'],
    'dataCriacao': datetime.today().date().isoformat(),
    'departamentoId': int(r.get('cod_agencia',0))
}, axis=1)

# Campos finais
base_princ['usuarioResponsavel'] = base_princ.get('gestor') if 'gestor' in base_princ.columns else ''
base_princ['usuarioSolicitante'] = USER
base_princ['statusChamado'] = 'RASCUNHO'
base_princ['numeroAgencia'] = base_princ['cod_agencia']
base_princ['nomeDepartamento'] = base_princ.get('nome_agencia') if 'nome_agencia' in base_princ.columns else ''
base_princ['departamentoId'] = base_princ['numeroAgencia']

cols_kanban = ['titulo','usuarioResponsavel','usuarioSolicitante','numeroAgencia','descricao','statusChamado','departamentoId','nomeDepartamento','dataNecessidade','categoriaChamado']
resultado_chamados = base_princ[cols_kanban].copy()

# Salvar
PARQUET_COMPLETO = os.path.join(RUTAS['saida_dir_giro'], 'giro_completo_principalidade.parquet')
PARQUET_KANBAN = os.path.join(RUTAS['saida_dir_giro'], 'giro_cards_kanban.parquet')
base_princ.to_parquet(PARQUET_COMPLETO, index=False)
resultado_chamados.to_parquet(PARQUET_KANBAN, index=False)
print({'completo': PARQUET_COMPLETO, 'kanban': PARQUET_KANBAN, 'linhas': len(resultado_chamados)})
resultado_chamados.head(3)

In [ ]:
path_dashboard = rf"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\associados_totais_tratados_dashboard.parquet"
if not os.path.exists(path_dashboard):
    raise FileNotFoundError("Dashboard não encontrado: " + path_dashboard)

assoc_dash = pd.read_parquet(path_dashboard, columns=['cpf_cnpj','nome_agencia','mc_total_1','gestor','ult_movimento'])
# Normalizar chave
def _norm_doc(v): 
    import re
    return re.sub(r'\D','', str(v)).lstrip('0')
assoc_dash['cpf_cnpj'] = assoc_dash['cpf_cnpj'].map(_norm_doc)

In [ ]:
# Guardar resultado da query em Parquet (fonte oficial)
import os, pandas as pd

saida_base = rf"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\associados_queda_principalidade.parquet"
tmp = saida_base + ".tmp"

# Executa a query (usa variável 'query' já definida acima) e grava diretamente
con.execute(f"COPY ({query}) TO '{tmp}' (FORMAT PARQUET, COMPRESSION ZSTD);")
os.replace(tmp, saida_base)
print("Parquet gerado:", saida_base)

# Verificação rápida de esquema / amostra
df_preview = pd.read_parquet(saida_base, engine="pyarrow")
print("Linhas gravadas:", len(df_preview), "| Colunas:", len(df_preview.columns))
print("Colunas:", df_preview.columns.tolist()[:12], "...")

In [ ]:
# Leitura para uso posterior
df = pd.read_parquet(rf"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\associados_queda_principalidade.parquet")
df.head()

In [ ]:
# Gerar e salvar parquet incluindo var_pontos, queda_flag, soma_quedas
import os, pandas as pd

saida = rf"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\associados_queda_principalidade.parquet"
tmp = saida + ".tmp"

# Executar a CTE (usa a variável 'query' já definida acima) e materializar
df_quedas = con.sql(query).fetchdf()

# Validar presença das colunas calculadas
esperadas_calc = ['var_pontos','queda_flag','soma_quedas']
faltan = [c for c in esperadas_calc if c not in df_quedas.columns]
if faltan:
    raise RuntimeError(f"Faltan columnas calculadas: {faltan}")

# Guardar atomicamente
df_quedas.to_parquet(tmp, index=False)
os.replace(tmp, saida)
print("Parquet salvo com colunas calculadas:", saida)
print("Total linhas:", len(df_quedas), "| Colunas:", len(df_quedas.columns))
print("Tem var_pontos?", 'var_pontos' in df_quedas.columns)

# Definir base_princ para seguir pipeline
base_princ = df_quedas.copy()
base_princ.head(2)

In [ ]:
# Montagem final COM filtros BBM e somente ativos (necessário antes de salvar)
from datetime import datetime, timedelta

if 'base_princ' not in globals():
    raise RuntimeError('base_princ não definido.')

# 1 Filtros obrigatórios (BBM + ativo)
riscos_permitidos = [r.upper() for r in PARAMS['filtros_risco_bbm']]
if 'nivel_risco' not in base_princ.columns:
    raise RuntimeError('Coluna nivel_risco ausente para filtro BBM.')
base_princ['nivel_risco_norm'] = base_princ['nivel_risco'].astype(str).str.upper().str.strip()
base_princ = base_princ[base_princ['nivel_risco_norm'].isin(riscos_permitidos)]

if 'flg_ativo' not in base_princ.columns:
    raise RuntimeError('Coluna flg_ativo ausente para filtro de ativos.')
base_princ = base_princ[base_princ['flg_ativo'].fillna(0)==1]

# 2 Merge dashboard (para gestor e nome_agencia) se ainda não aplicado
if 'assoc_dash' in globals():
    if 'cpf_cnpj' in base_princ.columns:
        base_princ['cpf_cnpj'] = base_princ['cpf_cnpj'].astype(str)
    base_princ = base_princ.merge(
        assoc_dash[['cpf_cnpj','gestor','nome_agencia']].drop_duplicates(),
        on='cpf_cnpj', how='left'
    )
else:
    raise RuntimeError('assoc_dash não carregado antes da montagem final.')

# 3 Derivar produtos se ainda não derivado
if 'prod_cat1_faltantes' not in base_princ.columns or 'prod_cat2_metricas' not in base_princ.columns:
    # Repetir lógica básica derivação rápida
    import numpy as np
    prod_cols = {
        'Chave PIX ativa':'cad_pix_ativo',
        'Débito automático':'debito_conta_ativo',
        'Cartão de débito':'possui_cartao_debito',
        'Cartão de crédito':'possui_cartao_credito',
        'Canais digitais':'transacao_app',
        'Credenciamento ativo':'possui_adquirencia',
        'Cobrança':'possui_cobranca',
        'Folha de Pagamento':'possui_folha_pagamento',
        'Domicílio':'possui_domicilio',
        'Open Finance (receptor)':'ativou_open_finance'
    }
    for c in prod_cols.values():
        if c in base_princ.columns:
            base_princ[c] = base_princ[c].replace({'S':1,'N':0}).fillna(0).astype(float)
    base_princ['prod_cat1_faltantes'] = base_princ.apply(
        lambda r: [n for n,c in prod_cols.items() if c in base_princ.columns and r.get(c,0)==0],
        axis=1
    )
    metricas2 = {
        'Cash-in':'cash_in',
        'Cash-out':'cash_out',
        'Total movimentações':'cash_total',
        'SOW Cartão':'sow_cartao'
    }
    base_princ['prod_cat2_metricas'] = base_princ.apply(
        lambda r: [f"{n}: {r.get(c)}" for n,c in metricas2.items() if c in base_princ.columns],
        axis=1
    )

# 4 Blocos
base_princ['bloco_isa'] = base_princ.apply(lambda r: (
    f"ISA atual: {r.get('isa_media')} | ISA média 3M: {r.get('isa_media_3m_calc'):.2f}"
        if pd.notnull(r.get('isa_media_3m_calc')) else
        (f"ISA atual: {r.get('isa_media')}" if pd.notnull(r.get('isa_media')) else '')
), axis=1)
base_princ['bloco_cat1'] = base_princ['prod_cat1_faltantes'].apply(
    lambda lst: 'Produtos básicos a ofertar: ' + ', '.join(lst) if lst else 'Produtos básicos completos')
base_princ['bloco_cat2'] = base_princ['prod_cat2_metricas'].apply(
    lambda lst: 'Indicadores fluxo / SOW: ' + '; '.join(lst))

# 5 Título e descrição
base_princ['titulo'] = base_princ.apply(lambda r: f"Oferta Principalidade - {r.get('cpf_cnpj')}", axis=1)
base_princ['descricao'] = base_princ.apply(lambda r: ' | '.join([
    f"Segmento: {r.get('segmento')}",
    r.get('bloco_isa',''),
    f"MC Total Atual: {r.get('cash_total')}",
    r.get('bloco_cat1',''),
    r.get('bloco_cat2','')
]), axis=1)

# 6 Categoria
base_princ['categoriaChamadoId'] = base_princ['prod_cat1_faltantes'].apply(lambda l: 1 if len(l)>0 else 2)
base_princ['nomeCategoria'] = base_princ['categoriaChamadoId'].map({1:'Acesso',2:'Fluxo de Caixa'})
base_princ['descricaoCategoria'] = base_princ['nomeCategoria'].map({
    'Acesso':'Ofertar Produtos e Serviços Básicos',
    'Fluxo de Caixa':'Ofertar Movimentações e Uso de Crédito'
})

# 7 Datas (divisão 4 partes)
base_princ = base_princ.sort_values('pontos_principalidade').reset_index(drop=True)
qtd = len(base_princ)
ini = datetime.today().date().replace(day=1)
semanas = [0,7,14,21]
base_princ['dataNecessidade'] = base_princ.index.map(
    lambda i: (ini + timedelta(days=semanas[(i * 4 // (qtd if qtd>0 else 1)) % 4])).isoformat() + 'T08:00:00.000Z'
)


In [ ]:
# Montagem final COM filtros BBM e somente ativos (necessário antes de salvar)
from datetime import datetime, timedelta

if 'base_princ' not in globals():
    raise RuntimeError('base_princ não definido.')

# 1 Filtros obrigatórios (BBM + ativo)
riscos_permitidos = [r.upper() for r in PARAMS['filtros_risco_bbm']]
if 'nivel_risco' not in base_princ.columns:
    raise RuntimeError('Coluna nivel_risco ausente para filtro BBM.')
base_princ['nivel_risco_norm'] = base_princ['nivel_risco'].astype(str).str.upper().str.strip()
base_princ = base_princ[base_princ['nivel_risco_norm'].isin(riscos_permitidos)]

if 'flg_ativo' not in base_princ.columns:
    raise RuntimeError('Coluna flg_ativo ausente para filtro de ativos.')
base_princ = base_princ[base_princ['flg_ativo'].fillna(0)==1]

# 2 Merge dashboard (para gestor e nome_agencia) se ainda não aplicado
if 'assoc_dash' in globals():
    if 'cpf_cnpj' in base_princ.columns:
        base_princ['cpf_cnpj'] = base_princ['cpf_cnpj'].astype(str)
    base_princ = base_princ.merge(
        assoc_dash[['cpf_cnpj','gestor','nome_agencia']].drop_duplicates(),
        on='cpf_cnpj', how='left'
    )
else:
    raise RuntimeError('assoc_dash não carregado antes da montagem final.')

# 3 Derivar produtos se ainda não derivado
if 'prod_cat1_faltantes' not in base_princ.columns or 'prod_cat2_metricas' not in base_princ.columns:
    # Repetir lógica básica derivação rápida
    import numpy as np
    prod_cols = {
        'Chave PIX ativa':'cad_pix_ativo',
        'Débito automático':'debito_conta_ativo',
        'Cartão de débito':'possui_cartao_debito',
        'Cartão de crédito':'possui_cartao_credito',
        'Canais digitais':'transacao_app',
        'Credenciamento ativo':'possui_adquirencia',
        'Cobrança':'possui_cobranca',
        'Folha de Pagamento':'possui_folha_pagamento',
        'Domicílio':'possui_domicilio',
        'Open Finance (receptor)':'ativou_open_finance'
    }
    for c in prod_cols.values():
        if c in base_princ.columns:
            base_princ[c] = base_princ[c].replace({'S':1,'N':0}).fillna(0).astype(float)
    base_princ['prod_cat1_faltantes'] = base_princ.apply(
        lambda r: [n for n,c in prod_cols.items() if c in base_princ.columns and r.get(c,0)==0],
        axis=1
    )
    metricas2 = {
        'Cash-in':'cash_in',
        'Cash-out':'cash_out',
        'Total movimentações':'cash_total',
        'SOW Cartão':'sow_cartao'
    }
    base_princ['prod_cat2_metricas'] = base_princ.apply(
        lambda r: [f"{n}: {r.get(c)}" for n,c in metricas2.items() if c in base_princ.columns],
        axis=1
    )

# 4 Blocos
base_princ['bloco_isa'] = base_princ.apply(lambda r: (
    f"ISA atual: {r.get('isa_media')} | ISA média 3M: {r.get('isa_media_3m_calc'):.2f}"
        if pd.notnull(r.get('isa_media_3m_calc')) else
        (f"ISA atual: {r.get('isa_media')}" if pd.notnull(r.get('isa_media')) else '')
), axis=1)
base_princ['bloco_cat1'] = base_princ['prod_cat1_faltantes'].apply(
    lambda lst: 'Produtos básicos a ofertar: ' + ', '.join(lst) if lst else 'Produtos básicos completos')
base_princ['bloco_cat2'] = base_princ['prod_cat2_metricas'].apply(
    lambda lst: 'Indicadores fluxo / SOW: ' + '; '.join(lst))

# 5 Título e descrição
base_princ['titulo'] = base_princ.apply(lambda r: f"Oferta Principalidade - {r.get('cpf_cnpj')}", axis=1)
base_princ['descricao'] = base_princ.apply(lambda r: ' | '.join([
    f"Segmento: {r.get('segmento')}",
    r.get('bloco_isa',''),
    f"MC Total Atual: {r.get('cash_total')}",
    r.get('bloco_cat1',''),
    r.get('bloco_cat2','')
]), axis=1)

# 6 Categoria
base_princ['categoriaChamadoId'] = base_princ['prod_cat1_faltantes'].apply(lambda l: 1 if len(l)>0 else 2)
base_princ['nomeCategoria'] = base_princ['categoriaChamadoId'].map({1:'Acesso',2:'Fluxo de Caixa'})
base_princ['descricaoCategoria'] = base_princ['nomeCategoria'].map({
    'Acesso':'Ofertar Produtos e Serviços Básicos',
    'Fluxo de Caixa':'Ofertar Movimentações e Uso de Crédito'
})

# 7 Datas (divisão 4 partes)
base_princ = base_princ.sort_values('pontos_principalidade').reset_index(drop=True)
qtd = len(base_princ)
ini = datetime.today().date().replace(day=1)
semanas = [0,7,14,21]
base_princ['dataNecessidade'] = base_princ.index.map(
    lambda i: (ini + timedelta(days=semanas[(i * 4 // (qtd if qtd>0 else 1)) % 4])).isoformat() + 'T08:00:00.000Z'
)

# 8 Objeto categoriaChamado
base_princ['categoriaChamado'] = base_princ.apply(lambda r: {
    'categoriaChamadoId': int(r['categoriaChamadoId']),
    'nomeCategoria': r['nomeCategoria'],
    'descricaoCategoria': r['descricaoCategoria'],
    'dataCriacao': datetime.today().date().isoformat(),
    'departamentoId': int(r.get('cod_agencia',0))
}, axis=1)

# 9 Campos finais (gestor pode estar ausente => fallback '')
base_princ['usuarioResponsavel'] = base_princ.get('gestor').fillna('') if 'gestor' in base_princ.columns else ''
base_princ['usuarioSolicitante'] = USER
base_princ['statusChamado'] = 'RASCUNHO'
base_princ['numeroAgencia'] = base_princ.get('cod_agencia')
base_princ['nomeDepartamento'] = base_princ.get('nome_agencia','')
base_princ['departamentoId'] = base_princ['numeroAgencia']

cols_saida = ['titulo','usuarioResponsavel','usuarioSolicitante','numeroAgencia','descricao',
              'statusChamado','departamentoId','nomeDepartamento','dataNecessidade','categoriaChamado']

resultado_chamados = base_princ[cols_saida].copy()

# 10 Salvar somente agora (depois filtros BBM + ativo)
parquet_completo = os.path.join(RUTAS['saida_dir_giro'], f'giro_completo_principalidade{data}.parrquet')
parquet_chamados = os.path.join(RUTAS['saida_dir_giro'], 'giro_chamados_principalidade.parquet')
base_princ.to_parquet(parquet_completo, index=False)
resultado_chamados.to_parquet(parquet_chamados, index=False)
print({'completo': parquet_completo, 'chamados': parquet_chamados,
       'linhas_chamados': len(resultado_chamados),
       'cols_finais': list(resultado_chamados.columns)})
resultado_chamados.head(1)

In [ ]:

con = duckdb.connect()

path = rf"C:\Users\{user}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\cia_pcp_indicador_principalidade_historico\\2025*.parquet"
query = rf'''
    CREATE TABLE dados AS
    SELECT * FROM read_parquet('{path}')
    
   
'''

con.execute(query)

In [ ]:

query = f''' 
      WITH tratados AS 
        (
            SELECT *,
            ROW_NUMBER() OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes DESC) AS rn,
            pontos_principalidade - LAG(pontos_principalidade)  OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC) AS var_pontos

            FROM dados
            QUALIFY rn <= 4
            ORDER BY ano_Mes
        ),
        
        marcacoes AS (
            SELECT  
            *,
            CASE WHEN var_pontos < 0 THEN 1 ELSE 0 END AS queda_flag
            FROM tratados 
            WHERE rn <= 3
            
        ),
        checagem AS (
            SELECT *,
            SUM(queda_flag) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes) AS soma_quedas
            FROM marcacoes
        )
        
        SELECT  * FROM checagem WHERE soma_quedas >= 3 and ano_mes = (SELECT MAX(ano_mes) FROM checagem)
        --SELECT  * FROM checagem WHERE cpf_cnpj = '00362741948'
        

        '''
con.sql(query).to_df()
       